# Generating Synthetic Data with LOGO-Programs, Descriptions and the respective Graphics

In [ ]:
# load
from _1_logo_pseudo_code_generator import generateLOGOPseudoCode
from _2_sampler import LOGOProgramSampler
from _3_executable_logo_primitives import ReGALLOGOPrimitives
from _4_logo_graphic_generator_v1 import PseudoProgramInterpreter as PseudoProgramInterpreter_v1
from _4_logo_graphic_generator_v2 import PseudoProgramInterpreter as PseudoProgramInterpreter_v2

generator=generateLOGOPseudoCode()

import json
import pandas as pd
import os

load the train and test data from the ReGAL-Paper. Join both and remove duplicate programs.

In [2]:
#LOGO
train_logo_data = "logo_data/python/train_200_dataset.jsonl"
test_logo_data = "logo_data/python/test_dataset.jsonl"

# Load train and test dataset
with open(f"../external/dependencies/{train_logo_data}", 'r') as f:
    train_data = [json.loads(line) for line in f]

with open(f"../external/dependencies/{test_logo_data}", 'r') as f:
    test_data = [json.loads(line) for line in f]

In [3]:
# Extract descriptions and programs from train_data and transform into pandas DataFrame
def extract_descriptions_and_programs(data):
    extracted_data = []
    for item in data:
        description = None
        program = None
        for message in item.get('messages', []):
            if message['from'] == 'human':
                description = message['value']
            elif message['from'] == 'gpt':
                program = message['value']
        if description and program:
            extracted_data.append([description, program])
    extracted_data = pd.DataFrame(extracted_data, columns=['Description', 'Program'])
    return extracted_data

# Extract descriptions and programs from train_data
df_train = extract_descriptions_and_programs(train_data)
df_test = extract_descriptions_and_programs(test_data)

#display(df_train)
#display(df_test)

# Append the test data to the train data
df_all = pd.concat([df_train, df_test], ignore_index=True)
#display(df_all)

#####################
# Drop all duplicate rows
df_all = df_all.drop_duplicates(subset=['Program']) # use the program as the unique identifier
#####################
display(df_all)

,Description,Program
0,4 concentric square s,"for i in range(5):\n embed(""""""for j in rang..."
1,6 sided snowflake with a medium line and a med...,"for j in range(6):\n embed(""""""forward(8)\nl..."
2,5 sided snowflake with a medium line and a sma...,"for j in range(5):\n embed(""""""forward(8)\nl..."
3,6 short line s in a row,"for j in range(6):\n embed(""""""forward(2)\nl..."
4,a small triangle connected by a big line to a ...,for i in range(3):\n forward(2)\n left(1...
...,...,...
306,8 sided snowflake with a medium circle and a s...,"for j in range(8):\n embed(""""""penup()\nforw..."
307,5 sided snowflake with 2 small circle s as arms,"for j in range(5):\n embed(""""""penup()\nforw..."
308,3 sided snowflake with a small square and a sm...,"for j in range(3):\n embed(""""""penup()\nforw..."
309,5 sided snowflake with a small 5 gon and a sma...,"for j in range(5):\n embed(""""""penup()\nforw..."


## Generate Graphics for the ReGAL dataset and the Synthetic Data

- initialize the sampler, if the train and test dataframes are porvided then the newly generated programs are not the same as the once in these datasets
- the data can be stored in json-line format
- next step is to initalize the interpreter, with this one can execute the programs and generate graphs
    - the interpreter generates .png-files
    - the size is for all graphics in one df the same (the aim is to keep size information which is relative to the other graphics thats why there can be a lot of whitespace this might be debatable)

In [ ]:
sampler=LOGOProgramSampler(generator, train_df=df_train, test_df=df_test) # this way only new programs are generated that are not in the train or test data

# Synthetic data
#synthetic_data = sampler.sample(50) 
#synthetic_data = pd.DataFrame(synthetic_data, columns=['Description', 'Program'])

#save data as json-line file with current timestamp
#timestamp = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")
#synthetic_data.to_json(f"data/synthetic_data_{timestamp}.jsonl", orient="records", lines=True)

# load synthetic data
synthetic_data = pd.read_json("data/synthetic_data_20250115181421.jsonl", orient="records", lines=True)

In [ ]:
# Generate graphics for the synthetic data 
interpreter = PseudoProgramInterpreter_v2()
interpreter.process_and_save_graphics(synthetic_data, output_dir="logo_graphic/synthetic_v2")

In [11]:
interpreter = PseudoProgramInterpreter_v1()
interpreter.process_and_save_graphics(synthetic_data, output_dir="logo_graphic/synthetic_v1") # here graphics are thigthly cropped around the image

In [8]:
print(synthetic_data['Description'].iloc[4])
print(synthetic_data['Program'].iloc[4])

print(synthetic_data['Description'].iloc[9])
print(synthetic_data['Program'].iloc[9])

7 medium line in a row
for j in range(7):
    embed("""forward(4)""", locals())
    penup()
    forward(2)
    left(0.0)

    pendown()
connected sequence of shapes: a small 6-gon, a short line, a small 9-gon
for i in range(6):
    forward(2)
    left(60.0)
forward(2)
for i in range(9):
    forward(2)
    left(40.0)


In [4]:
# TEST 
# create a test dataset form the all_data
test_indices = [98, 44, 100, 99, 200, 212, 214, 201, 53, 54, 282] # examples representing different shapes and combinations for testing purposes
df_test_subset = df_all.loc[test_indices].reset_index(drop=True)
#display(df_test_subset)

interpreter = PseudoProgramInterpreter_v2()
interpreter.process_and_save_graphics(df_test_subset, output_dir="logo_graphic/11testshapes")

In [5]:
# ReGAL DATA GRAPHICS
interpreter = PseudoProgramInterpreter_v2()
interpreter.process_and_save_graphics(df_all, output_dir="logo_graphic/train200_test")